#### This notebook is used for extracting and formatting data for publishing.

In [1]:
import sys
sys.path.extend(['../commons'])

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
import pandas as pd
import os
import re
import codecs
from IPython.display import display, Math, Latex
from IPython.display import Image as ipImage
from six.moves import cPickle as pickle
import string
from PIL import Image
import numpy as np
import h5py
import matplotlib as mpl
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [4]:
pd.options.display.max_rows = 50
pd.options.display.max_colwidth = 600
pd.options.display.expand_frame_repr = False
pd.options.display.colheader_justify = 'left'

In [5]:
import data_commons as dtc
import dl_commons as dlc
import viz_commons as viz
from viz_commons import VisualizeDir, DiffParams, VisualizeStep

Using TensorFlow backend.


### Load results of test run

In [8]:
# storedir = '/zpool_3TB/i2l/tb_metrics/2017-12-21 02-20-10 PST 140K_score_89.0/test_runs/step_00168100_score89.0_publish/store_2'
storedir = '/zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09/test_runs/step_00167526_score89.0_publish/store_2'
clobber = False
dump = False

original_image_dir = '/zpool_3TB/i2l/data/dataset5/formula_images'
rel_dumpdir = 'eval_images'
dumpdir = os.path.join(storedir, rel_dumpdir)
rendered_dir = os.path.join(storedir, rel_dumpdir, 'rendered_images')

In [9]:
vd = VisualizeDir(os.path.expanduser(storedir))
last_step = vd.view_steps()[1][-1]
print('last_step = %d' % last_step)
vs = VisualizeStep(vd, 'test', last_step)
df, df_result, df_data, sr_label = vs.get_preds(rel_dumpdir=rel_dumpdir, dump=dump, clobber=clobber)
# df = vs.dump_preds(rel_dumpdir=rel_dumpdir, dump=dump)

Loaded /zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09/test_runs/step_00167526_score89.0_publish/store_2/hyper.pkl and /zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09/test_runs/step_00167526_score89.0_publish/store_2/args.pkl
Loaded ../data/dataset5/training_56/df_train.pkl (114408, 3)
Loaded ../data/dataset5/training_56/df_test.pkl (14280, 3)
Loaded ../data/dataset5/training_56/df_valid.pkl (14280, 3)
num eval_steps = 1
eval_steps = [(167526, 146585)]
all_steps = []
last_step = 167526


### View and save the unmatched images

In [10]:
with open(os.path.join(rendered_dir, 'unmatched_filenames.txt'), 'r') as f:
    files = []
    for fname in f:
        files.append(os.path.basename(fname.strip()))
    
df_bad = df.loc[files]
targets=[]; preds=[]; index=[]
for row in df_bad[['target_seq', 'pred_seq']].sample(55).itertuples(index=True):
    index.append(row[0])
    targets.append('$%s$'%row[1])
    preds.append('$%s$'%row[2])
df_ = pd.DataFrame(data={'gold': targets, 'pred': preds})
df_

,gold,pred
0,"$\hat { T } _ { 5 5 } = | \Lambda _ { B } | - \frac { | \Lambda _ { B } | } { \mathrm { c o s h } ^ { 2 } \left( \sqrt { \frac { 2 \hat { \kappa } ^ { 2 } } { 3 } | \Lambda _ { B } | } \, y \right) } \, . $",$\hat { T } _ { 5 5 } = | \Lambda _ { B } | - \frac { 1 } { \operatorname { c o s h } ^ { 2 } \left( \sqrt { \frac { 2 k ^ { 2 } } { k } | \Lambda _ { B } | y } \right) } \cdot $
1,"$\omega \equiv \frac { \epsilon _ { 4 } - \epsilon _ { 3 } } { 2 p } L \, . $",$\omega = \frac { \epsilon _ { 4 } - \epsilon _ { 3 } } { 2 p } L . $
2,"$\sqrt { - \tilde { \hat { g } } } \ = \ \hat { G } _ { \underline { { x } } \underline { { x } } } ^ { - 1 } \, s q r t { - \hat { g } } \, . $","$\sqrt { - \tilde { g } } \ = \ \hat { G } _ { \mathrm { g e s } } ^ { - 1 } \, s q r t - \hat { g } \, . $"
3,$\overrightarrow { \not \! \! D } _ { \nu R } \equiv \frac { 1 } { 2 } V _ { a } ^ { \mu } \gamma ^ { a } \left( \vec { \partial } _ { \mu } + \frac { 1 } { 2 } \omega _ { \mu } ^ { c d } \sigma _ { c d } \right) $,$\overline { { \not \! \! D } } _ { \nu R } \equiv \frac { 1 } { 2 } V _ { a } ^ { \mu } \gamma ^ { a } \left( \vec { \partial } _ { \mu } + \frac { 1 } { 2 } \omega _ { \mu } ^ { c d } \sigma _ { c d } \right) $
4,"$c _ { m } ^ { i } ( \xi _ { 1 } , \xi _ { 2 } ) = K _ { m } ^ { i ~ p _ { 1 } , \ldots , p _ { \alpha } ~ q _ { 1 } , \ldots , q _ { \beta } } \, \xi _ { 1 p _ { 1 } } , \ldots , \xi _ { 1 p _ { \alpha } } \, \xi _ { 2 q _ { 1 } } , \ldots , \xi _ { 2 q _ { \beta } } \, . $","$c _ { m } ^ { i } ( \xi _ { 1 } , \xi _ { 2 } ) = K _ { m } ^ { i \ p _ { 1 } , . . . , p _ { \alpha } \ \ q _ { 1 } , . . . q _ { q } } \, \xi _ { 1 p _ { 1 } } , \ldots , \xi _ { 1 p _ { \alpha } } \, \xi _ { 2 q _ { 1 } } , \ldots , \xi _ { 2 q _ { \beta } } \, . $"
5,"$\sum _ { i s o s p i n \; \; I _ { 1 } , I _ { 2 } } \sigma _ { \pi \pi \rightarrow \pi \pi \pi \pi } ( s , I _ { 1 } , I _ { 2 } ) = \frac { 6 7 } { 2 ^ { 1 7 } \; 3 ^ { 4 } \; \pi ^ { 5 } } \; \; \frac { s ^ { 3 } } { F _ { \pi } ^ { 8 } } ~ ~ ~ ~ ~ ~ ~ F _ { \pi } = 9 3 ~ ~ \mathrm { M e V } $","$\sum _ { i s o s i n _ { l } } A _ { I _ { 2 } } \sigma _ { \pi \pi \rightarrow \pi \pi \pi } ( s , I _ { 1 } , I _ { 2 } ) = \frac { 6 7 } { 2 ^ { 1 7 } \; 3 ^ { 4 } \; \pi ^ { 5 } } \; \; \frac { s ^ { 3 } } { F _ { \pi } ^ { 8 } } \; \; \; \; \; \; \; F _ { \pi } = 9 3 \; \; \mathrm { M e V } $"
6,"$\begin{array} { c c } { \vec { x } _ { a } = \vec { x } _ { C M } + y _ { a } \hat { n } \, , ~ ~ ~ ~ } & { ~ ~ ~ \displaystyle { \sum _ { a } } \, y _ { a } = 0 \, . } \\ \end{array} $","$\vec { x } _ { a } = \vec { x } C _ { M M } + y _ { a } \hat { n } \, , \qquad \qquad \sum _ { a } y _ { a } = 0 \, . $"
7,$| a \rangle = \sum _ { j \in { \cal E } } \frac { \psi _ { a } ^ { j } } { \sqrt { S _ { 1 j } } } | j \rangle \! \rangle $,$| a \rangle = \sum _ { j \in { \bf \xi } } \frac { \psi _ { a } ^ { j } } { \sqrt { S _ { 1 j } } } | j \rangle $
8,"$Z = \int \prod _ { x } \mathrm { d } A \, \mathrm { d } \psi \, \mathrm { d } { \bar { \psi } } \, e ^ { \mathrm { i } \, \int \mathrm { d } ^ { 2 } x \, { \cal L } } , $","$Z = \int \prod _ { x } \mathrm { d } A \, \mathrm { d } \psi \, \mathrm { d } \bar { \psi } \, e ^ { \mathrm { i } \, f ^ { 3 } x \, { \cal L } } , $"
9,"$F ( \omega + i 0 ^ { + } ) - F ( \omega - i 0 ^ { + } ) = 2 \pi i \sigma ( \omega ) \, | F ( \omega + i 0 ^ { + } ) | ^ { 2 } \, , $","$F ( \omega + i 0 ^ { + } ) - F ( \omega - i 0 ^ { + } ) = 2 \pi i \sigma ( \omega ) \left| F ( \omega + i 0 ^ { + } ) \right| ^ { 2 } , $"


### In favor of compatibility with publishing tools, remove entries that are not rendered by MathJax 
Though they rendered fine with render_latex.py above, MathJax has difficulty rendering them. Therefore we will remove them for visualization purposes just in case it causes a problem with publishing tools downstream.

In [13]:
df_2_ = df_.drop([18, 32, 53]).reset_index(drop=True)[:50]
df_2_

,gold,pred
0,"$\hat { T } _ { 5 5 } = | \Lambda _ { B } | - \frac { | \Lambda _ { B } | } { \mathrm { c o s h } ^ { 2 } \left( \sqrt { \frac { 2 \hat { \kappa } ^ { 2 } } { 3 } | \Lambda _ { B } | } \, y \right) } \, . $",$\hat { T } _ { 5 5 } = | \Lambda _ { B } | - \frac { 1 } { \operatorname { c o s h } ^ { 2 } \left( \sqrt { \frac { 2 k ^ { 2 } } { k } | \Lambda _ { B } | y } \right) } \cdot $
1,"$\omega \equiv \frac { \epsilon _ { 4 } - \epsilon _ { 3 } } { 2 p } L \, . $",$\omega = \frac { \epsilon _ { 4 } - \epsilon _ { 3 } } { 2 p } L . $
2,"$\sqrt { - \tilde { \hat { g } } } \ = \ \hat { G } _ { \underline { { x } } \underline { { x } } } ^ { - 1 } \, s q r t { - \hat { g } } \, . $","$\sqrt { - \tilde { g } } \ = \ \hat { G } _ { \mathrm { g e s } } ^ { - 1 } \, s q r t - \hat { g } \, . $"
3,$\overrightarrow { \not \! \! D } _ { \nu R } \equiv \frac { 1 } { 2 } V _ { a } ^ { \mu } \gamma ^ { a } \left( \vec { \partial } _ { \mu } + \frac { 1 } { 2 } \omega _ { \mu } ^ { c d } \sigma _ { c d } \right) $,$\overline { { \not \! \! D } } _ { \nu R } \equiv \frac { 1 } { 2 } V _ { a } ^ { \mu } \gamma ^ { a } \left( \vec { \partial } _ { \mu } + \frac { 1 } { 2 } \omega _ { \mu } ^ { c d } \sigma _ { c d } \right) $
4,"$c _ { m } ^ { i } ( \xi _ { 1 } , \xi _ { 2 } ) = K _ { m } ^ { i ~ p _ { 1 } , \ldots , p _ { \alpha } ~ q _ { 1 } , \ldots , q _ { \beta } } \, \xi _ { 1 p _ { 1 } } , \ldots , \xi _ { 1 p _ { \alpha } } \, \xi _ { 2 q _ { 1 } } , \ldots , \xi _ { 2 q _ { \beta } } \, . $","$c _ { m } ^ { i } ( \xi _ { 1 } , \xi _ { 2 } ) = K _ { m } ^ { i \ p _ { 1 } , . . . , p _ { \alpha } \ \ q _ { 1 } , . . . q _ { q } } \, \xi _ { 1 p _ { 1 } } , \ldots , \xi _ { 1 p _ { \alpha } } \, \xi _ { 2 q _ { 1 } } , \ldots , \xi _ { 2 q _ { \beta } } \, . $"
5,"$\sum _ { i s o s p i n \; \; I _ { 1 } , I _ { 2 } } \sigma _ { \pi \pi \rightarrow \pi \pi \pi \pi } ( s , I _ { 1 } , I _ { 2 } ) = \frac { 6 7 } { 2 ^ { 1 7 } \; 3 ^ { 4 } \; \pi ^ { 5 } } \; \; \frac { s ^ { 3 } } { F _ { \pi } ^ { 8 } } ~ ~ ~ ~ ~ ~ ~ F _ { \pi } = 9 3 ~ ~ \mathrm { M e V } $","$\sum _ { i s o s i n _ { l } } A _ { I _ { 2 } } \sigma _ { \pi \pi \rightarrow \pi \pi \pi } ( s , I _ { 1 } , I _ { 2 } ) = \frac { 6 7 } { 2 ^ { 1 7 } \; 3 ^ { 4 } \; \pi ^ { 5 } } \; \; \frac { s ^ { 3 } } { F _ { \pi } ^ { 8 } } \; \; \; \; \; \; \; F _ { \pi } = 9 3 \; \; \mathrm { M e V } $"
6,"$\begin{array} { c c } { \vec { x } _ { a } = \vec { x } _ { C M } + y _ { a } \hat { n } \, , ~ ~ ~ ~ } & { ~ ~ ~ \displaystyle { \sum _ { a } } \, y _ { a } = 0 \, . } \\ \end{array} $","$\vec { x } _ { a } = \vec { x } C _ { M M } + y _ { a } \hat { n } \, , \qquad \qquad \sum _ { a } y _ { a } = 0 \, . $"
7,$| a \rangle = \sum _ { j \in { \cal E } } \frac { \psi _ { a } ^ { j } } { \sqrt { S _ { 1 j } } } | j \rangle \! \rangle $,$| a \rangle = \sum _ { j \in { \bf \xi } } \frac { \psi _ { a } ^ { j } } { \sqrt { S _ { 1 j } } } | j \rangle $
8,"$Z = \int \prod _ { x } \mathrm { d } A \, \mathrm { d } \psi \, \mathrm { d } { \bar { \psi } } \, e ^ { \mathrm { i } \, \int \mathrm { d } ^ { 2 } x \, { \cal L } } , $","$Z = \int \prod _ { x } \mathrm { d } A \, \mathrm { d } \psi \, \mathrm { d } \bar { \psi } \, e ^ { \mathrm { i } \, f ^ { 3 } x \, { \cal L } } , $"
9,"$F ( \omega + i 0 ^ { + } ) - F ( \omega - i 0 ^ { + } ) = 2 \pi i \sigma ( \omega ) \, | F ( \omega + i 0 ^ { + } ) | ^ { 2 } \, , $","$F ( \omega + i 0 ^ { + } ) - F ( \omega - i 0 ^ { + } ) = 2 \pi i \sigma ( \omega ) \left| F ( \omega + i 0 ^ { + } ) \right| ^ { 2 } , $"


In [23]:
mingled = []
for row in df_2_.itertuples(index=False):
    mingled.extend(row)
index_tuples = [(i, l) for i in range(len(mingled)/2) for l in ('X', 'Y')]
index = pd.MultiIndex.from_tuples(index_tuples)
df_3_ = pd.DataFrame(data={'LaTeX': mingled}, index=index)
df_3_

LaTeX                                                                                                                                                                                                                                                                                                                                                                                                                              
0  X                                                                                                                                                                                                                      $\hat { T } _ { 5 5 } = | \Lambda _ { B } | - \frac { | \Lambda _ { B } | } { \mathrm { c o s h } ^ { 2 } \left( \sqrt { \frac { 2 \hat { \kappa } ^ { 2 } } { 3 } | \Lambda _ { B } | } \, y \right) } \, . $
   Y                                                                                                                                                                                                                                                  $\hat { T } _ { 5 5 } = | \Lambda _ { B } | - \frac { 1 } { \operatorname { c o s h } ^ { 2 } \left( \sqrt { \frac { 2 k ^ { 2 } } { k } | \Lambda _ { B } | y } \right) } \cdot $
1  X                                                                                                                                                                                                                                                                                                                                                       $\omega \equiv \frac { \epsilon _ { 4 } - \epsilon _ { 3 } } { 2 p } L \, . $
   Y                                                                                                                                                                                                                                                                                                                                                               $\omega = \frac { \epsilon _ { 4 } - \epsilon _ { 3 } } { 2 p } L . $
2  X                                                                                                                                                                                                                                                                                    $\sqrt { - \tilde { \hat { g } } } \ = \ \hat { G } _ { \underline { { x } } \underline { { x } } } ^ { - 1 } \, s q r t { - \hat { g } } \, . $
   Y                                                                                                                                                                                                                                                                                                                         $\sqrt { - \tilde { g } } \ = \ \hat { G } _ { \mathrm { g e s } } ^ { - 1 } \, s q r t - \hat { g } \, . $
3  X                                                                                                                                                                                                              $\overrightarrow { \not \! \! D } _ { \nu R } \equiv \frac { 1 } { 2 } V _ { a } ^ { \mu } \gamma ^ { a } \left( \vec { \partial } _ { \mu } + \frac { 1 } { 2 } \omega _ { \mu } ^ { c d } \sigma _ { c d } \right) $
   Y                                                                                                                                                                                                                $\overline { { \not \! \! D } } _ { \nu R } \equiv \frac { 1 } { 2 } V _ { a } ^ { \mu } \gamma ^ { a } \left( \vec { \partial } _ { \mu } + \frac { 1 } { 2 } \omega _ { \mu } ^ { c d } \sigma _ { c d } \right) $
4  X                                                                                                                                                 $c _ { m } ^ { i } ( \xi _ { 1 

In [25]:
pd.options.display.latex.repr = True
with open('/tmp/df_2_frag.tex', 'w') as f:
    f.write(df_3_.to_latex(escape=False))

# End

In [ ]:
num2display = 10
# with open(os.path.join(rendered_dir, 'unmatched_filenames.txt'), 'r') as f:
#     files = np.random.choice(f.readlines(), num2display)
#     for fname in files:
#         fname = os.path.basename(fname.strip())
#         display(df.loc[fname][['target_seq', 'pred_seq']])
#         gold = ipImage(filename=os.path.join(rendered_dir, 'images_gold', fname), unconfined=True)
#         pred = ipImage(filename=os.path.join(rendered_dir, 'images_pred', fname), unconfined=True)
#         display(gold)
#         display(pred)
gold = []; pred = [];
gold_images = []; pred_images = []
with open(os.path.join(rendered_dir, 'unmatched_filenames.txt'), 'r') as f:
    files = np.random.choice(f.readlines(), num2display)
    for fname in files:
        fname = os.path.basename(fname.strip())
        if not os.path.exists(os.path.join(rendered_dir, 'images_pred', fname)):
            continue
        gold_images.append(ipImage(filename=os.path.join(rendered_dir, 'images_gold', fname), format='png', unconfined=True))
        gold.append('<img src=".%s"/>' % os.path.join(rendered_dir, 'images_gold', fname))
        pred_images.append(ipImage(filename=os.path.join(rendered_dir, 'images_pred', fname), format='png', unconfined=True))
        pred.append('<img src=".%s"/>' % os.path.join(rendered_dir, 'images_pred', fname))
df_= pd.DataFrame({'gold': gold, 'pred': pred})
HTML((df_).to_html(escape=False))

In [ ]:
def loadImage(path, mode="RGB"):
    img = Image.open(path)
    img = img.convert(mode)
    return np.asarray(img)

def plotImage(path, axes):
    image_data = loadImage(path)
#     title = os.path.splitext(os.path.basename(path))[0] + ' %s'%(image_data.shape,)
#     axes.set_title( title )
    axes.set_xlim(0,1000)
    axes.imshow(image_data, aspect='equal', extent=None, resample=False, interpolation='bilinear')
    return

def plotImages(image_paths, dpi=None):
    """ image_details should be an array of image path and width - [[path, image_data, image_data.shape], ...] """
    ## Display image data loaded as a numpy array
    try:
        plt.close(fig)
    except:
        pass
    orig_dpi = plt.rcParams['figure.dpi']

    for path in (image_paths):
        gold = os.path.join(rendered_dir, 'images_gold', path)
        pred = os.path.join(rendered_dir, 'images_pred', path)
        image_data.append(plotImage(gold, grid[i]))
        i+=1
        image_data.append(plotImage(pred, grid[i]))
        i+=1
        
    with mpl.rc_context(rc={'figure.dpi': dpi or orig_dpi}):
        fig = plt.figure(figsize=(20.,2.*len(image_paths)))
        grid = ImageGrid(fig, 111, nrows_ncols=(len(image_paths),2), axes_pad=(0.1, 0.5), label_mode="L")
        i = 0
        for path in (image_paths):
            gold = os.path.join(rendered_dir, 'images_gold', path)
            pred = os.path.join(rendered_dir, 'images_pred', path)
            plotImage(gold, grid[i])
            i+=1
            plotImage(pred, grid[i])
            i+=1
    return # image_data

num2display = 5
with open(os.path.join(rendered_dir, 'unmatched_filenames.txt'), 'r') as f:
    files = [os.path.basename(fname.strip()) for fname in np.random.choice(f.readlines(), num2display)]
fnames=[]
plotImages(files)

# fig, axes = plt.subplots(nrows=num2display, ncols=2, sharex=True, sharey=True, squeeze=True)
# for i, fname in enumerate(files):
#     fname = os.path.basename(fname.strip())
#     im_gold = loadImage(os.path.join(rendered_dir, 'images_gold', fname))
#     im_pred = loadImage(os.path.join(rendered_dir, 'images_pred', fname))
#     axes[i][0].imshow(im_gold, aspect='equal', extent=None, resample=False, interpolation='bilinear', cmap='gray')
#     axes[i][1].imshow(im_pred, aspect='equal', extent=None, resample=False, interpolation='bilinear', cmap='gray')

In [ ]:
# for fname in files:
#     print os.path.join(rendered_dir,'images_gold', os.path.basename(fname.strip()))
#     print os.path.join(rendered_dir,'images_pred', os.path.basename(fname.strip()))